# 3d mitochondria visualizer with synapses
This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409611,2864896,3878784,93134,59860,836,93000,59406,798,93294,60294,867,648518346349536888,333419.72,214298.80,33440
2409612,2161774,4075748,80312,56522,1814,79536,55348,1714,80940,57054,1889,648518346349524070,287516.96,202348.76,72560
2409613,2753701,4576444,90324,60436,2037,89490,60120,1851,90750,61138,2142,648518346341354380,323359.92,216360.88,81480
2409614,1963708,5805612,75674,72546,744,75266,71628,670,76032,73302,795,648518346343047176,270912.92,259714.68,29760


# Enter a cellid of interest

In [6]:
# pull all mitos from a cellid of interest
cellid = 648518346349539853

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf

# interesting cellids
# 648518346349530724 microglia with 149 mitos
# 648518346349527319 astrocyte containing the largest mito in the volume and several additional large mitos
# 648518346349537555 apical dendrite likely from a L5 or L6 pyr neuron
# 648518346349537741 pyr neuron with largest mito in the volume for a neuron
# 648518346349508279 oligodendrocyte
# 648518346349539853 one of three neurons in the default view of Neuroglancer

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
1906,3098978,30152,95960,67402,621,95872,67374,614,96072,67434,631,648518346349539853,343536.80,241299.16,24840
3625,3170590,125152,97918,50832,954,97666,50600,937,98176,51040,968,648518346349539853,350546.44,181978.56,38160
4857,3970091,332188,114540,58920,1279,114014,58698,1272,114964,59126,1287,648518346349539853,410053.20,210933.60,51160
6770,3620264,1832860,109466,53436,650,107652,52536,535,111472,54400,714,648518346349539853,391888.28,191300.88,26000
7055,3515725,1089960,106290,59742,1079,105440,58126,968,106992,61020,1173,648518346349539853,380518.20,213876.36,43160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405530,3175427,697804,98004,52030,677,97756,51616,641,98292,52408,729,648518346349539853,350854.32,186267.40,27080
2406629,2264009,768908,82692,50680,689,81984,50518,573,83166,50866,786,648518346349539853,296037.36,181434.40,27560
2406830,3068752,785008,96824,52592,156,96696,52254,114,96970,53020,221,648518346349539853,346629.92,188279.36,6240
2407304,3511865,836376,104410,56030,2010,103808,55838,1891,105332,56156,2125,648518346349539853,373787.80,200587.40,80400


In [7]:
mito_query_mitolist = mito_querydf.mito_id.to_list()
print(f"length: "+str(len(mito_query_mitolist)))
print(f"type: "+str(type(mito_query_mitolist)))
print('')
# print(mito_query_mitolist) # uncomment to print the mito id list

length: 945
type: <class 'list'>



# vtk 3d viewer

In [8]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://seungdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [9]:
cell_id = cellid

In [10]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [ ]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

In [11]:
len(mito_query_mitolist)

945

In [12]:
mito_query_mitolist[0]

3098978

### The following cell only needs to be run once
- You do not need to run this cell if you have previously downloaded the mito mesh files
- Note: this will take a long time, which gets longer the more mitos are in the cell of interest

In [ ]:
# download the mito meshes for this cell

for i in range(len(mito_query_mitolist)):
    mito_id = mito_query_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [13]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator = {}
for i in range(len(mito_query_mitolist)):
    mito_seg_id = mito_query_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator['mito_mesh_' + str(i)] = mito_mm.mesh(filename = mito_mesh_file)
    
locals().update(var_iterator)

In [14]:
cell_opac = 0.075
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000025DBB075B00) at 0x0000025D8E133468>

# Add synapses to visualization
- Download synapse table from https://www.microns-explorer.org/phase1

In [15]:
syn = pd.read_csv('data/221206_pni_synapses_v185.csv')
syn

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,2477272,648518346349538285,648518346349536759,1103,280784,261896,40600,70050,65534,1015,70196,65474,1015,70104,65602,1015
1,4538436,648518346342404863,648518346342414574,62,365832,292808,25800,91472,73196,645,91458,73202,645,91396,73216,645
2,1124039,648518346342921567,648518346345323296,62,229144,155952,48080,57250,39028,1202,57286,38988,1202,57312,38966,1203
3,5245264,648518346342796822,648518346349537255,955,398452,208560,50760,99538,52184,1276,99613,52140,1269,99550,52098,1276
4,6928613,648518346342800412,648518346348190542,718,461868,245212,53560,115462,61350,1335,115467,61303,1339,115468,61296,1335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239270,2167340,648518346348045782,648518346349537840,106,268776,173356,14360,67190,43302,361,67194,43339,359,67216,43364,360
3239271,1944654,648518346347834064,648518346349537840,50,267344,156520,8520,66762,39152,213,66836,39130,213,66874,39134,213
3239272,2749491,648518346347876810,648518346349537840,139,292772,156680,12600,73136,39170,315,73193,39170,315,73218,39160,315
3239273,6924909,648518346348343883,648518346349537840,987,462768,237776,8520,115680,59370,208,115692,59444,213,115628,59430,214


In [16]:
# filter all the pre- and post- synapses for this cell
cell_post_subgraph = syn.query(f'post_root_id=={seg_id}') #this shows all post-syn sites on this seg_id cell
cell_pre_subgraph = syn.query(f'pre_root_id=={seg_id}') #this one shows all pre-syn sites on the axon of seg_id cell

# make an array of xyz synapse positions
postsyn_xyz=cell_post_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
presyn_xyz=cell_pre_subgraph[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values

In [17]:
len(postsyn_xyz)

4588

In [18]:
len(presyn_xyz)

104

In [19]:
# Make a list of the pre_root_ids that form a synapse on this cell
cell_post_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
105577,5423743,648518346349539853,648518346349539853,141,400968,189528,8800,100258,47418,219,100242,47382,220,100214,47348,219
483721,4173119,648518346346292396,648518346349539853,275,351568,166176,4720,87830,41574,118,87892,41544,118,87886,41520,117
526247,6154007,648518346344073003,648518346349539853,71,430440,282608,13880,107608,70680,349,107610,70652,347,107634,70604,347
526251,4840445,648518346349512517,648518346349539853,163,381380,209084,34880,95300,52292,865,95345,52271,872,95340,52218,865
526252,5258468,648518346349380254,648518346349539853,65,395248,222416,23480,98826,55586,588,98812,55604,587,98808,55634,586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154287,5034925,648518346342928499,648518346349539853,703,386028,201484,66240,96460,50300,1656,96507,50371,1656,96450,50388,1656
3154288,5237014,648518346341406355,648518346349539853,566,392472,204696,40280,98048,51090,1005,98118,51174,1007,98000,51166,1005
3154289,5409705,648518346348203681,648518346349539853,103,401752,174564,57120,100376,43678,1428,100438,43641,1428,100452,43614,1428
3154307,5830148,648518346349373893,648518346349539853,147,418744,193576,42400,104690,48446,1060,104686,48394,1060,104688,48362,1060


In [20]:
# Uncomment if you want to generate a list for copy and pasting
# cell_post_subgraph["pre_root_id"].to_list()

In [21]:
# Make a list of the post_root_ids that receive a synapse from this cell
cell_pre_subgraph

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
3612,6120354,648518346349539853,648518346349537577,51,428064,257576,10240,107036,64442,256,107016,64394,256,106976,64362,256
11224,5717046,648518346349539853,648518346349538272,175,410400,263080,15880,102624,65824,396,102600,65770,397,102548,65734,396
19070,5330828,648518346349539853,648518346344077270,221,396080,270356,83960,98988,67534,2100,99020,67589,2099,98992,67636,2100
32987,5110553,648518346349539853,648518346342795643,74,387240,268712,23960,96798,67234,598,96810,67178,599,96812,67150,598
50767,2657264,648518346349539853,648518346341367473,66,287552,249976,9480,71906,62468,236,71888,62494,237,71848,62536,236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939870,5230493,648518346349539853,648518346349539864,610,398240,200632,17520,99576,50130,435,99560,50158,438,99516,50166,435
2956421,1779838,648518346349539853,648518346349539560,411,256584,191720,72840,64116,47882,1820,64146,47930,1821,64114,47952,1820
3125130,2865676,648518346349539853,648518346341391953,172,294896,250236,13960,73664,62576,348,73724,62559,349,73728,62566,346
3125495,5337960,648518346349539853,648518346341355539,130,391032,281288,71920,97746,70366,1796,97758,70322,1798,97748,70298,1796


In [22]:
# Uncomment if you want to generate a list for copy and pasting
# cell_pre_subgraph["post_root_id"].to_list()

In [23]:
# settings to visualize pre and post synaptic sites (as dots)
pre_color = (0.2, 0.9, 0.2)
post_color = (0.9, 0.2, 0.2)
pre_opac = 0.25
post_opac = 0.25
pre_size = 250
post_size = 200
presyn_actor = trimesh_vtk.point_cloud_actor(presyn_xyz, size=pre_size, opacity=pre_opac, color=pre_color)  
postsyn_actor = trimesh_vtk.point_cloud_actor(postsyn_xyz, size=post_size, opacity=post_opac, color=post_color)

# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.05
cell_color = (0.2, 0.4, 0.7)
mito_opac = 0.35

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes

actor_iterator = {}
for i in range(len(mito_query_mitolist)):
    random_color = list(np.random.random(size=3))
    actor_iterator['mito_mesh_actor_' + str(i)] = trimesh_vtk.mesh_actor(var_iterator['mito_mesh_'+str(i)], opacity=mito_opac, color=(random_color))
    
locals().update(actor_iterator)

# update dictionary for the render actors code below
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator['presyn_actor'] = presyn_actor
actor_iterator['postsyn_actor'] = postsyn_actor

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=150)

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)



setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000025DC4208490) at 0x0000025D801031C8>